# Import Libraries

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
anime_df = pd.read_csv('anime.csv')

In [26]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [27]:
rating_df = pd.read_csv('rating.csv')

In [28]:
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


# Data Shapes and Info

In [29]:
print(f"anime set (row, col): {anime_df.shape}\n\nrating set (row, col): {rating_df.shape}")

anime set (row, col): (12294, 7)

rating set (row, col): (7813737, 3)


In [30]:
print('Anime:\n')
anime_df.info()
print('\n', '*'*50 ,'\nRating:\n')
rating_df.info()

Anime:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB

 ************************************************** 
Rating:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


# Handling missing values

In [31]:
print('Anime missing values %:\n')
print(round(anime_df.isnull().sum().sort_values(ascending=False)/len(anime_df.index), 4) * 100)
print('\n', '*'*50, '\nRating missing values %: \n')
print(round(rating_df.isnull().sum().sort_values(ascending=False)/len(rating_df), 4) * 100)

Anime missing values %:

rating      1.87
genre       0.50
type        0.20
anime_id    0.00
name        0.00
episodes    0.00
members     0.00
dtype: float64

 ************************************************** 
Rating missing values %: 

user_id     0.0
anime_id    0.0
rating      0.0
dtype: float64


In [32]:
print(anime_df['genre'].mode())
print(anime_df['type'].mode())

0    Hentai
dtype: object
0    TV
dtype: object


In [33]:
anime_df = anime_df[~np.isnan(anime_df['rating'])]

anime_df['genre'] = anime_df['genre'].fillna(anime_df['genre'].dropna().mode().values[0])

anime_df['type'] = anime_df['type'].fillna(anime_df['type'].dropna().mode().values[0])

anime_df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

# Feature Engineering

In [34]:
rating_df['rating'] = rating_df['rating'].apply(lambda x: np.nan if x==-1 else x)
rating_df.head()

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


In [35]:
anime_df = anime_df[anime_df['type']=='TV']

rated_anime = rating_df.merge(anime_df, left_on = 'anime_id', right_on = 'anime_id', suffixes = ['_user', ''])

rated_anime = rated_anime[['user_id', 'name', 'rating']]

rated_anime_7500 = rated_anime[rated_anime.user_id <= 7500]
rated_anime_7500.head()

,user_id,name,rating
0,1,Naruto,7.81
1,3,Naruto,7.81
2,5,Naruto,7.81
3,6,Naruto,7.81
4,10,Naruto,7.81


## Pivot table for similarity

In [36]:
pivot = rated_anime_7500.pivot_table(index=['user_id'], columns=['name'], values='rating')
pivot.head()

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.11,NaN


In [37]:
import scipy as sp
from scipy import sparse

pivot_n = pivot.apply(lambda x: (x-np.mean(x)) / (np.max(x) - np.min(x)), axis=1)

pivot_n.fillna(0, inplace=True)

pivot_n = pivot_n.T

pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]

piv_sparse = sparse.csr_matrix(pivot_n.values)

# Cosine Similarity Model

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

anime_similarity = cosine_similarity(piv_sparse)

ani_sim_df = pd.DataFrame(anime_similarity, index=pivot_n.index, columns = pivot_n.index)

In [40]:
def anime_recommendation(ani_name):
    number = 1
    print('Recommended because you watched {}:\n'.format(ani_name))
    for anime in ani_sim_df.sort_values(by = ani_name, ascending = False).index[1:6]:
        print(f'#{number}: {anime}, {round(ani_sim_df[anime][ani_name]*100,2)}% match')
        number +=1

In [46]:
anime_recommendation('Sword Art Online')

Recommended because you watched Sword Art Online:

#1: Guilty Crown, 33.83% match
#2: Ao no Exorcist, 33.42% match
#3: Another, 30.93% match
#4: Akame ga Kill!, 30.91% match
#5: Naruto, 30.03% match
